In [1]:
println("init")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.248:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1600685088472)
SparkSession available as 'spark'


init


In [34]:
import spark.implicits._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.{StringIndexer, VectorIndexer, IndexToString}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{DecisionTreeClassifier, LogisticRegression, NaiveBayes, RandomForestClassifier}

import spark.implicits._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.{StringIndexer, VectorIndexer, IndexToString}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{DecisionTreeClassifier, LogisticRegression, NaiveBayes, RandomForestClassifier}


In [4]:
val iris = spark.read.format("csv")
                  .option("sep", ",")
                  .option("inferSchema", "true")
                  .option("header", "true")
                  .load("/home/leon/data/iris.csv")

iris: org.apache.spark.sql.DataFrame = [sepal_length: double, sepal_width: double ... 3 more fields]


In [13]:
iris.schema.map(_.dataType)

res8: Seq[org.apache.spark.sql.types.DataType] = List(DoubleType, DoubleType, DoubleType, DoubleType, StringType)


In [27]:
val assembler = new VectorAssembler()
        .setInputCols(Array("sepal_length", "sepal_width", "petal_length", "petal_width"))
        .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_1295dc8148b5, handleInvalid=error, numInputCols=4


In [26]:
iris.schema.names

res15: Array[String] = Array(sepal_length, sepal_width, petal_length, petal_width, species)


In [29]:
val transformed_data = assembler.transform(iris).drop("sepal_length", "sepal_width", "petal_length", "petal_width")


transformed_data: org.apache.spark.sql.DataFrame = [species: string, features: vector]


In [36]:
val labelIndexer = new StringIndexer()
        .setInputCol("species")
        .setOutputCol("indexedSpecies")
        .fit(transformed_data)
        
val featureIndexer = new VectorIndexer()
        .setInputCol("features")
        .setOutputCol("indexedFeatures")
        .setMaxCategories(4)
        .fit(transformed_data)
        
// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
        .setInputCol("prediction")
        .setOutputCol("predictedLabel")
        .setLabels(labelIndexer.labels)
        
// Declaring ML Pipeline 
val pipeline = new Pipeline()

labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_58fd09a27971, handleInvalid=error
featureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = VectorIndexerModel: uid=vecIdx_eb15290b58cc, numFeatures=4, handleInvalid=error
labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_ad74cfcdf12f
pipeline: org.apache.spark.ml.Pipeline = pipeline_9c1ab826392c


In [40]:

// Train a DecisionTree
val dt: DecisionTreeClassifier = new DecisionTreeClassifier()
        .setLabelCol("indexedSpecies")
        .setFeaturesCol("indexedFeatures")
pipeline.setStages(Array(labelIndexer, featureIndexer, dt, labelConverter))


// Train the Logistic Regression.
val lr: LogisticRegression = new LogisticRegression()
        .setLabelCol("indexedSpecies")
        .setMaxIter(10)
        .setRegParam(0.3)
        .setElasticNetParam(0.8)
        .setFamily("multinomial")

pipeline.setStages(Array(labelIndexer, featureIndexer, lr, labelConverter))


// Train the Naive Bayes
val nb: NaiveBayes = new NaiveBayes().setLabelCol("indexedSpecies")
pipeline.setStages(Array(labelIndexer, featureIndexer, nb, labelConverter))


val rf: RandomForestClassifier = new RandomForestClassifier()
        .setLabelCol("indexedSpecies")
        .setFeaturesCol("indexedFeatures")
        .setNumTrees(10)

pipeline.setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_2760f71f8189
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_805275f027f0
nb: org.apache.spark.ml.classification.NaiveBayes = nb_9977c8cae93b
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_4db4134ce205
res21: pipeline.type = pipeline_9c1ab826392c


In [41]:
val model = pipeline.fit(transformed_data)

// Saving Model to HDFS
model.save("/home/leon/projects/epsilon-ensemble/models/iris_model")

model: org.apache.spark.ml.PipelineModel = pipeline_9c1ab826392c
